# Module 4 - Assignment 3 - Creating AI Enabled Systems

Trystan May

Started: Sept. 24, 2022

In [1]:
import pandas as pd
import typing as tp
import nltk
import os

In [2]:
path_to_data: str = os.path.join('..', 'data', 'Assign3', 'Musical_instruments_reviews.csv')

os.path.exists(path_to_data)

True

In [3]:
df: pd.DataFrame = pd.read_csv(path_to_data)
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"
...,...,...,...,...,...,...,...,...,...
10256,A14B2YH83ZXMPP,B00JBIVXGC,Lonnie M. Adams,"[0, 0]","Great, just as expected. Thank to all.",5.0,Five Stars,1405814400,"07 20, 2014"
10257,A1RPTVW5VEOSI,B00JBIVXGC,Michael J. Edelman,"[0, 0]",I've been thinking about trying the Nanoweb st...,5.0,"Long life, and for some players, a good econom...",1404259200,"07 2, 2014"
10258,AWCJ12KBO5VII,B00JBIVXGC,Michael L. Knapp,"[0, 0]",I have tried coated strings in the past ( incl...,4.0,Good for coated.,1405987200,"07 22, 2014"
10259,A2Z7S8B5U4PAKJ,B00JBIVXGC,"Rick Langdon ""Scriptor""","[0, 0]","Well, MADE by Elixir and DEVELOPED with Taylor...",4.0,Taylor Made,1404172800,"07 1, 2014"


In [4]:
df['tokenized'] = df['summary'].apply(lambda s: nltk.word_tokenize(s))
df['tokenized']

0                                                   [good]
1                                                   [Jake]
2                               [It, Does, The, Job, Well]
3                      [GOOD, WINDSCREEN, FOR, THE, MONEY]
4         [No, more, pops, when, I, record, my, vocals, .]
                               ...                        
10256                                        [Five, Stars]
10257    [Long, life, ,, and, for, some, players, ,, a,...
10258                               [Good, for, coated, .]
10259                                       [Taylor, Made]
10260    [These, strings, are, really, quite, good, ,, ...
Name: tokenized, Length: 10261, dtype: object

In [5]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

df['stemmed'] = df['summary'].apply(lambda s: stemmer.stem(s))

df[['summary', 'stemmed']]

,summary,stemmed
0,good,good
1,Jake,jake
2,It Does The Job Well,it does the job wel
3,GOOD WINDSCREEN FOR THE MONEY,good windscreen for the money
4,No more pops when I record my vocals.,no more pops when i record my vocals.
...,...,...
10256,Five Stars,five star
10257,"Long life, and for some players, a good econom...","long life, and for some players, a good econom..."
10258,Good for coated.,good for coated.
10259,Taylor Made,taylor mad


That honestly seems like it randomly stems things that don't need it. Like 'vocals.' and 'cable'. I can see that it didn't stem it because of the period in vocals and -able *is* a common ending of words cable becoming c makes sense but still

In [6]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

# Got the idea for this code here - https://gaurav5430.medium.com/using-nltk-for-lemmatizing-sentences-c1bfff963258

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

lemmatizer = WordNetLemmatizer()

def convert_to_wordnet_pos(nltk_pos: str) -> tp.Optional[str]:
    if nltk_pos.startswith('J'):
        return wn.ADJ
    elif nltk_pos.startswith('V'):
        return wn.VERB
    elif nltk_pos.startswith('N'):
        return wn.NOUN
    elif nltk_pos.startswith('R'):
        return wn.ADV
    else:
        return None


def lemmatize_sentence(sentence: str) -> str:
    tokenized: tp.List[str] = nltk.wordpunct_tokenize(sentence.lower())
    nltk_poses: tp.List[tp.Tuple[str, str]] = nltk.pos_tag(tokenized)
    wordnet_poses: tp.List[tp.Tuple[str, str]] = [(token, convert_to_wordnet_pos(nltk_pos)) for token, nltk_pos in nltk_poses]

    lemmatized_sentence: tp.List[str] = [lemmatizer.lemmatize(token, pos=wn_pos) if wn_pos is not None else token for token, wn_pos in wordnet_poses]

    return ' '.join(lemmatized_sentence)


df['lemmatized'] = df['summary'].apply(lambda s: lemmatize_sentence(s))

df[['summary', 'stemmed', 'lemmatized']]

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/duke_trystan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/duke_trystan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/duke_trystan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,summary,stemmed,lemmatized
0,good,good,good
1,Jake,jake,jake
2,It Does The Job Well,it does the job wel,it do the job well
3,GOOD WINDSCREEN FOR THE MONEY,good windscreen for the money,good windscreen for the money
4,No more pops when I record my vocals.,no more pops when i record my vocals.,no more pop when i record my vocal .
...,...,...,...
10256,Five Stars,five star,five star
10257,"Long life, and for some players, a good econom...","long life, and for some players, a good econom...","long life , and for some player , a good econo..."
10258,Good for coated.,good for coated.,good for coat .
10259,Taylor Made,taylor mad,taylor make


In [7]:
for i, (summary, lemmatized, stemmed, tokenized) in df[['summary', 'lemmatized', 'stemmed', 'tokenized']].head().iterrows():
    print(f'Row #{i:02d}:')
    print(f'\tSummary: {summary}')
    tokenized_str: str = '", "'.join(tokenized)
    print(f'\tTokenized: ["{tokenized_str}"]')
    print(f'\tStemmed: {stemmed}')
    print(f'\tLemmatized: {lemmatized}')


Row #00:
	Summary: good
	Tokenized: ["good"]
	Stemmed: good
	Lemmatized: good
Row #01:
	Summary: Jake
	Tokenized: ["Jake"]
	Stemmed: jake
	Lemmatized: jake
Row #02:
	Summary: It Does The Job Well
	Tokenized: ["It", "Does", "The", "Job", "Well"]
	Stemmed: it does the job wel
	Lemmatized: it do the job well
Row #03:
	Summary: GOOD WINDSCREEN FOR THE MONEY
	Tokenized: ["GOOD", "WINDSCREEN", "FOR", "THE", "MONEY"]
	Stemmed: good windscreen for the money
	Lemmatized: good windscreen for the money
Row #04:
	Summary: No more pops when I record my vocals.
	Tokenized: ["No", "more", "pops", "when", "I", "record", "my", "vocals", "."]
	Stemmed: no more pops when i record my vocals.
	Lemmatized: no more pop when i record my vocal .
